<a href="https://colab.research.google.com/github/wolfdnb/Projeto-x/blob/main/Projeto_da_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes_(Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents=' vistos em portugal ',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['requisitos para visto em portugal', 'tipos de vistos para portugal']
Páginas utilizadas na resposta: vfsglobal.com, www2.gov.pt, mne.gov.pt, eduportugal.eu, nomadglobal.com, cidadania4u.com.br, cibtvisas.com.br, eduportugal.eu



In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar. Você deve utilizar a ferramenta google_maps para mostar locais relevantes .
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você usa as últimas tendências de forma criativa
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco  em Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Visto portugal
Maravilha! Vamos então criar o post sobre novidades em Visto portugal

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para fornecer as informações mais recentes e relevantes sobre os vistos de Portugal, farei algumas pesquisas no Google. Seguem as perguntas que guiarão minha busca:
> 
>  *  Quais são os tipos de vistos de Portugal mais relevantes e recentes para 2025?
>  *  Quais são as últimas notícias sobre mudanças ou atualizações nos requisitos de visto para Portugal?
>  *  Como solicitar um visto para Portugal em 2025?
>  *  Quais são os vistos de Portugal mais populares entre brasileiros em 2025?
> 
>  
> Com base nas informações mais recentes disponíveis, aqui estão os principais pontos sobre vistos para Portugal em 2025:
> 
>  **1. Tipos de Vistos Mais Relevantes:**
> 
>  *   **Visto de Estada Temporária:** Para estadias de até um ano, ideal para estudos, trabalho temporário, estágio ou tratamento médico.
>  *   **Visto de Residência:** Para quem pretende morar em Portugal por mais de um ano. Inclui opções como:
>   *   **Visto de Trabalho (D1 ou D3):** Necessário ter um contrato de trabalho ou promessa de contrato.
>   *   **Visto de Estudante (D4):** Para quem vai estudar em Portugal (ensino superior, intercâmbio, etc.).
>   *   **Visto de Empreendedor (D2):** Para quem deseja abrir um negócio ou trabalhar como freelancer.
>   *   **Visto de Rendas Próprias (D7):** Para quem possui rendimentos estáveis (aposentadoria, etc.).
>   *   **Visto para Nómadas Digitais:** Para quem trabalha remotamente e tem uma renda mensal comprovada.
>   *   **Visto para Procura de Trabalho:** Permite residir em Portugal por até 6 meses para procurar emprego.
>   *   **Visto de Reagrupamento Familiar (D6):** Para familiares de residentes em Portugal.
> 
>  **2. Brasileiros Precisam de Visto?**
> 
>  *   **Não precisam para turismo:** Brasileiros não precisam de visto para estadias de até 90 dias para turismo ou negócios.
>  *   **Precisam para morar, trabalhar ou estudar:** Para estadias mais longas, é necessário solicitar o visto apropriado antes de viajar.
> 
>  **3. Como Solicitar um Visto:**
> 
>  *   **Pelo VFS Global:** O processo de solicitação é feito através da VFS Global, empresa terceirizada que representa o governo português no Brasil.
>  *   **Documentos Comuns:**
>   *   Formulário de solicitação preenchido
>   *   Fotos 3x4
>   *   Passaporte válido
>   *   Seguro viagem ou PB4
>   *   Certidão de antecedentes criminais
>   *   Comprovantes de renda e acomodação
>  *   **Passo a Passo:**
>   1.  Acesse o site da VFS Global.
>   2.  Preencha o formulário online.
>   3.  Reúna os documentos necessários.
>   4.  Pague as taxas.
>   5.  Agende e compareça ao centro de solicitação para entregar a documentação.
> 
>  **4. ETIAS em 2026:**
> 
>  *   A partir de 2026, brasileiros isentos de visto para estadias curtas precisarão obter uma autorização de viagem ETIAS (Sistema Europeu de Informação e Autorização de Viagem).
> 
>  **5. Vistos Populares entre Brasileiros:**
> 
>  *   Além dos vistos de trabalho e estudo, o visto para procura de trabalho tem sido bastante procurado por brasileiros que desejam tentar a sorte no mercado de trabalho português.
>  *   O visto D7 (rendas próprias) também é popular entre aposentados e pessoas com rendimentos passivos.
> 
>  **6. Taxas de Visto:**
> 
>  *   O Visto Schengen custa em torno de R$ 779,58 (incluindo taxas consulares, de transferência e de processamento). Os preços podem variar.
> 
>  **7. Dicas Importantes:**
> 
>  *   Solicite o visto com antecedência (pelo menos 30 dias antes da viagem para vistos de estada temporária e 60 dias para vistos de residência).
>  *   Reúna toda a documentação com cuidado e preencha o formulário corretamente.
>  *   Não compre passagens aéreas antes de ter o visto aprovado.
> 
>  **8. Atrasos e Mudanças:**
> 
>  *   Há relatos de atrasos na emissão de vistos, com alguns casos ultrapassando 200 dias de espera.
>  *   O governo português tem tomado medidas para agilizar o processo, como a contratação de mais funcionários para os consulados.
>  *   Mudanças nas regras de imigração têm exigido que o visto seja solicitado antes da entrada no país.
> 
>  **9. Acordo para Agilizar Vistos de Trabalho:**
> 
>  *   Em abril de 2025, foi assinado um protocolo para agilizar a contratação de trabalhadores estrangeiros, com o objetivo de conceder vistos em 20 dias, desde que cumpridos os requisitos legais.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Com base nas informações coletadas, o tema mais relevante para um post sobre vistos para Portugal é:
> 
> **Tema:** Vistos para Portugal em 2025: Guia Completo e Atualizado para Brasileiros
> 
> **Pontos Relevantes:**
> 
> *   **Tipos de Vistos:** Detalhar os vistos mais procurados por brasileiros (Trabalho, Estudo, Empreendedor, Rendas Próprias, Nómadas Digitais, Procura de Trabalho, Reagrupamento Familiar).
> *   **Processo de Solicitação:** Explicar o passo a passo para solicitar o visto através da VFS Global, incluindo documentos necessários e taxas.
> *   **Prazos e Atrasos:** Abordar os prazos para solicitação, os relatos de atrasos e as medidas que o governo português tem tomado para agilizar o processo.
> *   **Novidades:** Informar sobre o acordo para agilizar vistos de trabalho e a necessidade de ETIAS a partir de 2026 para estadias curtas.
> *   **Dicas:** Oferecer dicas práticas para evitar problemas na solicitação, como solicitar o visto com antecedência e reunir toda a documentação corretamente.
> 
> **Plano para o Post:**
> 
> 1.  **Título Atraente:** "Vistos para Portugal em 2025: O Guia Definitivo para Brasileiros Realizarem o Sonho de Morar na Europa" ou algo similar.
> 2.  **Introdução:**
>     *   Contextualizar a crescente busca por vistos para Portugal por brasileiros.
>     *   Mencionar que o post visa fornecer informações atualizadas e relevantes sobre o processo.
>     *   Informar que brasileiros não precisam de visto para turismo de até 90 dias.
> 3.  **Tipos de Vistos:**
>     *   Explicar detalhadamente cada um dos vistos mais relevantes (D1, D2, D3, D4, D6, D7, Nómadas Digitais, Procura de Trabalho).
>     *   Indicar para quem cada tipo de visto é adequado e quais os requisitos específicos.
> 4.  **Como Solicitar o Visto:**
>     *   Passo a passo detalhado para solicitar o visto através da VFS Global.
>     *   Lista completa dos documentos necessários para cada tipo de visto.
>     *   Informações sobre as taxas e como efetuar o pagamento.
> 5.  **Prazos e Atrasos:**
>     *   Alertar sobre os possíveis atrasos na emissão de vistos.
>     *   Informar sobre as medidas que o governo português tem tomado para agilizar o processo.
>     *   Sugerir que os interessados solicitem o visto com a maior antecedência possível.
> 6.  **Novidades:**
>     *   Informar sobre o acordo para agilizar vistos de trabalho (mencionar o prazo de 20 dias).
>     *   Alertar sobre a necessidade de ETIAS a partir de 2026 para estadias curtas.
> 7.  **Dicas Importantes:**
>     *   Solicitar o visto com antecedência (pelo menos 30 dias para estada temporária e 60 dias para residência).
>     *   Reunir toda a documentação com cuidado e preencher o formulário corretamente.
>     *   Não comprar passagens aéreas antes de ter o visto aprovado.
>     *   Verificar os horários de atendimento dos consulados e da VFS Global.
>     *   Buscar informações em fontes oficiais (site do governo português, VFS Global).
> 8.  **Conclusão:**
>     *   Reforçar a importância de se planejar com antecedência e buscar informações precisas.
>     *   Incentivar os leitores a compartilharem suas experiências e dúvidas nos comentários.
> 9.  **Call to Action:**
>     *   Convidar os leitores a se inscreverem na newsletter para receberem atualizações sobre vistos e imigração.
>     *   Oferecer um e-book gratuito com um guia completo sobre vistos para Portugal.
> 
> 
> Ótimo! O plano para o post está bem detalhado. Aqui estão algumas sugestões adicionais para enriquecer o conteúdo e torná-lo ainda mais útil para os leitores:
> 
> *   **Vídeos:**
>     *   Incluir vídeos explicativos sobre cada tipo de visto, com entrevistas de especialistas ou depoimentos de pessoas que já passaram pelo processo.
>     *   Criar um vídeo mostrando o passo a passo da solicitação do visto na VFS Global.
> *   **Checklists:**
>     *   Disponibilizar checklists em PDF para download, com a lista completa de documentos necessários para cada tipo de visto.
> *   **Links Úteis:**
>     *   Adicionar links para os sites oficiais da VFS Global, do governo português (AIMA), e para outros recursos relevantes.
> *   **Casos Práticos:**
>     *   Apresentar casos práticos de pessoas que obtiveram sucesso na solicitação do visto, destacando os pontos-chave de cada caso.
> *   **Atualizações:**
>     *   Manter o post sempre atualizado com as últimas notícias e mudanças nas regras de imigração.
>     *   Criar um sistema de alertas para notificar os leitores sobre novas informações.
> 
> Com essas adições, o post se tornará uma referência completa e confiável sobre vistos para Portugal em 2025, ajudando muitos brasileiros a realizarem o sonho de morar na Europa.
> 


--------------------------------------------------------------


...

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-80' coro=<AsyncClient.aclose() done, defined at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 406, in aclose
    await self._pool.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 345, in _close_connections
    await connection.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 173, in aclose
    await self._connection.aclose()
  File "/usr/local/lib/pytho

--------------------------------------------------------------


Exception in thread Thread-35 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/u


--- 📝 Resultado do Agente 4 (Revisor) ---



--------------------------------------------------------------
